# M2608.001300 Machine Learning<br> Assignment #5 Final Projects (Pytorch)


Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them.

**For understanding of this work, please carefully look at given PPT file.**

Note: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

In [13]:
#from google.colab import drive
#drive.mount('/content/drive')

import os
import random

import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
from PIL import Image
import matplotlib.pyplot as plt
import resnet

Load datasets


In [14]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
NONE = ['NONE'] # label for empty space
ALL_CHAR_SET = NUMBER + ALPHABET + NONE
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 7

#print(ALL_CHAR_SET_LEN)
#print(ALL_CHAR_SET.index('NONE'))
#print(ALL_CHAR_SET)

def encode(a):
    onehot = [0]*ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] += 1
    return onehot

# modified dataset class
class Mydataset(Dataset):
    def __init__(self, img_path, label_path, is_train=True, transform=None):
        self.path = img_path
        self.label_path = label_path
        if is_train: 
            self.img = os.listdir(self.path)[:3]
            #self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:10000]
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:3]
        else: 
            self.img = os.listdir(self.path)[:1000]
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:1000]
        
        self.transform = transform
        self.max_length = MAX_CAPTCHA
        
    def __getitem__(self, idx):
        img_path = self.img[idx]
        img = Image.open(f'{self.path}/{self.img[idx]}')
        # convert to gray mode
        img = img.convert('L')
        label = self.labels[idx]
        #print(label)
        label_oh = []
        
        # one-hot for each character
                
        for i in range(self.max_length):
            if i < len(label):
                label_oh += encode(label[i])
            else:
                #label_oh += [0]*ALL_CHAR_SET_LEN
                label_oh += encode('NONE')
            
        if self.transform is not None:
            img = self.transform(img)
        #print(img.shape)
        #print(type(label_oh))
        #print(label)
        return img, np.array(label_oh), label
    def __len__(self):
        return len(self.img)


    
#data size : 160*60
transform = transforms.Compose([
    transforms.Resize([160, 60]),
    transforms.ToTensor()

#TODO
    #ColorJitter
    #RandomAffine
    #randomapply
    #...
])



In [5]:
for step, i in enumerate(train_dl):
        img, label_oh, label = i
        print(img.shape)
        print(label_oh.shape)
        #print(label_oh)
        print(label)


torch.Size([3, 1, 160, 60])
torch.Size([3, 259])
('yib8', '2vb', 'ho4')


In [15]:
"""Loading DATA"""
# Change to your own data folder path!
gPath = './'

train_ds = Mydataset(gPath+'Data/train/', gPath+'Data/train.txt',transform=transform)
test_ds = Mydataset(gPath+'Data/test/', gPath+'Data/test.txt',False, transform)
train_dl = DataLoader(train_ds, batch_size=128, num_workers=4)
test_dl = DataLoader(test_ds, batch_size=1, num_workers=4)

In [16]:
"""To CUDA for local run"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
#GPUID = '4' # define GPUID
#os.environ["CUDA_VISIBLE_DEVICES"] = str(GPUID)


True


Problem 1: Design LSTM model for captcha image recognition. (10 points)

In [22]:
#TODO 1 : stack lstm layers
#TODO 2 : bidirectional RNN (since this captcha data has no sequential meanings)
#https://pytorch.org/docs/master/generated/torch.nn.LSTM.html

#embed_dim=10

class LSTM(nn.Module):
    def __init__(self, cnn_dim, hidden_size, vocab_size, num_layers=1):
        super(LSTM, self).__init__()
        '''
        # define the properties
        self.cnn_dim = cnn_dim
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        # lstm cell
        self.lstm_cell = nn.LSTMCell(input_size=self.vocab_size, hidden_size=hidden_size)
    
        # output fully connected layer
        self.fc_in = nn.Linear(in_features=self.cnn_dim, out_features=self.vocab_size)
        self.fc_out = nn.Linear(in_features=self.hidden_size, out_features=self.vocab_size)
    
        # embedding layer
        #self.embed = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.vocab_size)
        self.embed = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=embed_dim)
        # activations
        self.softmax = nn.Softmax(dim=1)
        #??
        '''
        # define properties
        self.cnn_dim = cnn_dim
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        # lstm module
        self.lstm = nn.LSTM(cnn_dim, hidden_size, bidirectional=True)
        self.embedding = nn.Linear(hidden_size*2, vocab_size)
        #self.lstm_cell = nn.LSTMCell()
        
        #fc layer for output
        self.fc_out = nn.Linear(hidden_size,vocab_size)
        
        #activation
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, features):
        
        outputs, _ = self.lstm(features)
        T, b, h = outputs.size()
        outputs = outputs.view(T*b,h)
        
        output = self.embedding(outputs)
        output = output.view(T,b,-1)
        
        
        '''
        batch_size = features.size(0)
        #print(batch_size)
        cnn_dim = features.size(1)
        #print(cnn_dim)
        #initialize h0,c0
        hidden_state = torch.zeros((batch_size, self.hidden_size)).cuda()
        cell_state = torch.zeros((batch_size, self.hidden_size)).cuda()
    
        # define the output tensor placeholder
        outputs = torch.empty((batch_size, captions.size(1), self.vocab_size)).cuda()

        # embed the captions
        captions_embed = self.embed(captions)
        '''
        #TODO
        
        return output




Problem 2: 

*   1.Connect CNN model to the designed LSTM model.
*   2.Replace ResNet to your own CNN model from Assignment3.


          


In [18]:
'''my betternet from AS3'''
class Inception(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(Inception, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )
    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1,y2,y3,y4], 1)
    
class BetterNet(nn.Module):
    def __init__(self):
        super(BetterNet, self).__init__()        
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7,stride=1,padding=3),
            nn.ReLU(True),
            nn.MaxPool2d(3,2,1)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=1,stride=1),
            nn.ReLU(True),
            nn.Conv2d(in_channels=64,out_channels=192,kernel_size=3,stride=1,padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(3,2,1)
        )
        self.incep_block1 = nn.Sequential(
            Inception(192,64,96,128,16,32,32),
            Inception(256,128,128,192,32,96,64),
            nn.MaxPool2d(3,2,1)
        )        
        self.incep_block2 = nn.Sequential(
            Inception(480,192,96,208,16,48,64),
            Inception(512,160,112,224,24,64,64),
            Inception(512,128,128,256,24,64,64),
            Inception(512,112,144,288,32,64,64),
            Inception(528,256,160,320,32,128,128),
            nn.MaxPool2d(3,2,1)
        )        
        self.incep_block3 = nn.Sequential(
            Inception(832,256,160,320,32,128,128),
            Inception(832,384,192,384,48,128,128),
            nn.AvgPool2d(kernel_size=4)
        )        
        self.fc_block = nn.Sequential(
            nn.Linear(1024,512),
            nn.ReLU(True),
            nn.Linear(512,ALL_CHAR_SET_LEN*MAX_CAPTCHA)
           # nn.ReLU(True),
           # nn.Linear(64,10)
        )        
    def forward(self, x):
        x=self.block1(x)
        x=self.block2(x)
        x=self.incep_block1(x)
        x=self.incep_block2(x)
        x=self.incep_block3(x)
        #x = x.view(-1,1024)
        #x=self.fc_block(x)
        out=x
        return out


In [29]:

"""ResNet

#CNN
betternet = resnet.resnet18(pretrained=False)
betternet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
betternet.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)
betternet = betternet.to(device)
"""


# CNN-RNN connected model

hidden_size=8
vocab_size=37

class CRNN(nn.Module):
    def __init__(self, hidden_size, vocab_size, num_layers=1):
        super(CRNN, self).__init__()
        self.cnn = BetterNet()
        self.LSTM = LSTM(1024,hidden_size,vocab_size,num_layers)
        
    def forward(self, x):
        conv_out = self.cnn(x)
        b,c,h,w = conv_out.size()
        conv_out = conv_out.squeeze(2) #h must be 1
        conv_out = conv_out.permute(2,0,1) #[w,b,c]
        
        rnn_out = self.LSTM(conv_out)
        output = F.log_softmax(output, dim=2)
        
        #rnn_in = conv_out.view(conv_out.size()[0],vocab_size,-1)
        #rnn_out = LSTM(rnn_in)
        
        return output

'''
# LSTM
cnn_dim=512 #resnet18-512
hidden_size=8
vocab_size=37 #ALL_CHAR_SET_LEN
lstm = LSTM(cnn_dim=cnn_dim, hidden_size=hidden_size, vocab_size=vocab_size)
lstm = lstm.to(device)
'''

crnn = CRNN(hidden_size,vocab_size,1)
crnn = crnn.to(device)

loss_func = nn.MultiLabelSoftMarginLoss()
crnn_optim = torch.optim.Adam(crnn.parameters(), lr=0.001)


'''
loss_func = nn.MultiLabelSoftMarginLoss()
cnn_optim = torch.optim.Adam(betternet.parameters(), lr=0.001)
'''


'\nloss_func = nn.MultiLabelSoftMarginLoss()\ncnn_optim = torch.optim.Adam(betternet.parameters(), lr=0.001)\n'

Problem3: Find hyper-parameters.


In [8]:
"""TRAINING"""
print_interval = 5
max_epoch = 1000

for epoch in range(max_epoch):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        img = Variable(img).cuda()
        label_oh = Variable(label_oh.float()).cuda()
        batch_size, _ = label_oh.shape
        
        #pred, feature = betternet(img)
        
        pred = crnn(img)
        
        pred_resized = pred.reshape([batch_size,1,-1])
        pred_resized = pred_resized.squeeze(1)
        
        
        loss = loss_func(pred_resized, label_oh)
        #cnn_optim.zero_grad()
        crnn_optim.zero_grad()
        loss.backward()
        crnn_optim.step()
        #cnn_optim.step() #TODO
        
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################

##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################
        if (step+1)%print_interval == 0:
            print('epoch:', epoch+1, 'step:', step+1, 'loss:', loss.item())

NameError: name 'betternet' is not defined

In [14]:
"""TEST"""
def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 
 


char_correct = 0
word_correct = 0
total = 0

betternet.eval()
lstm.eval()

with torch.no_grad():
    for step, (img, label_oh, label) in enumerate(test_dl):
        char_count =0
        img = Variable(img).cuda()
        label_oh = Variable(label_oh.float()).cuda()
        pred, feature = betternet(img) #TODO

        label_len = label[0]
        pred = pred.squeeze(0)
        label_oh = label_oh.squeeze(0)
        
        c0,c1,c2,c3,c4,c5,c6 = get_char_count(pred.squeeze()) 
        d0,d1,d2,d3,d4,d5,d6 = get_char_count(label_oh) 
         
        c = '%s%s%s%s%s%s%s' % (c0, c1, c2, c3, c4, c5, c6)
        d = '%s%s%s%s%s%s%s' % (d0, d1, d2, d3, d4, d5, d6)
    
        char_count += (c0==d0)+(c1==d1)+(c2==d2)+(c3==d3)+(c4==d4)+(c5==d5)+(c6==d6)
        char_correct += char_count

        if(bool(str(label[0]) in str(c))):
            word_correct+=1

        total += 1
       
print(100/7*char_correct/total)
print(100*word_correct/total)
"""END TEST"""

NameError: name 'betternet' is not defined